# Задача регрессии на реальных данных
Рассмотрим аналог Boston Housing Dataset с ценами на жильё в Москве за 2011-2015 год, данные предоставлены [Сбербанком](https://www.kaggle.com/c/sberbank-russian-housing-market/data), спасибо им за это.

А ещё:
- научимся работать с категориальными (номинальными) признаками
- поймём, почему важно нормализовать данные до того, как отправлять их считаться в модель
- посмотрим на альтернативы sklearn'овской LinearRegression

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

На том же сайте есть [пример](https://www.kaggle.com/captcalculator/a-very-extensive-sberbank-exploratory-analysis) хорошего первичного анализа. Откроем таблицу:

In [ ]:
data = pd.read_csv('train.csv')
data.head()

Как видим, признаков достаточно много (292), так что излюбленный метод `.info()` особо не поможет:

In [ ]:
data.info()

Посмотрим на количество пропусков по столбцам:

In [ ]:
data.isnull().sum()

Как видим, про некоторые признаки данных не очень много, так что выберем колонки, смысл которых мы заведомо будем понимать. 

In [ ]:
target_column = 'price_doc'

In [ ]:
nominal_feature_columns = [
    'sub_area',
    'culture_objects_top_25',
    'ecology',
]

In [ ]:
data[nominal_feature_colums].isnull().sum()

In [ ]:
internal_feature_columns = [
    'full_sq',
    'life_sq',
    'kitch_sq',
    'floor',
    'max_floor',
    'num_room',
]

In [ ]:
data[internal_feature_columns].isnull().sum()

In [ ]:
school_feature_columns = [
    'children_preschool',
    'preschool_education_centers_raion',
    'children_school',
    'school_education_centers_raion', 
    'school_education_centers_top_20_raion',
    'university_top_20_raion',
    'additional_education_raion',
    'additional_education_km',
    'university_km',
]

In [ ]:
data[school_feature_columns].isnull().sum()

In [ ]:
infrastructure_feature_columns = [
    'nuclear_reactor_km',
    'thermal_power_plant_km',
    'power_transmission_line_km',
    'incineration_km',
    'water_treatment_km',
    'railroad_station_walk_km',
    'railroad_station_walk_min',
    'railroad_station_avto_km',
    'railroad_station_avto_min',
    'public_transport_station_km',
    'public_transport_station_min_walk',
    'water_km',
    'mkad_km',
    'ttk_km',
    'sadovoe_km',
    'bulvar_ring_km',
    'kremlin_km',
]

In [ ]:
data[infrastructure_feature_columns].isnull().sum()

Посмотрели на колонки и пропуски в них, теперь объединим всё в один список и будем использовать в качестве признаков только перечисленные столбцы:

In [ ]:
feature_columns = nominal_feature_colums + internal_feature_columns + school_feature_columns + infrastructure_feature_columns + [target_column]

Сохраните таблицу только с перечисленными колонками и удалите из неё все строки с пропусками:

In [ ]:
# YOUR CODE

## OHE aka One Hot Encoding
Что сейчас мешает просто взять и запустить LinearRegression? Дело в том, что три колонки содержат не понятные компьютеру числа, а текстовые значения, а именно:

In [ ]:
data[nominal_feature_colums]

Есть простой способ закодировать данные. Например, рассмотрим колонку `ecology`. Она принимает только следующие значения:

In [ ]:
np.unique(data.ecology)

Есть два варианта действий:
- просто заменить каждое значение на число (4 - 'excellent', 3 - 'good' etc.)
- рассмотреть каждое значение как отдельный признак и создать в таблице новые колонки с названиями этих значений

Подумайте, почему $1$й способ хорошо подойдёт для колонки `ecology`, а $2$й -- для других двух столбцов.

Создайте словарь, сопоставляющий числовые значения от $0$ до $4$ описанию экологии от `'no data'` до `'excellent'` соответственно:

In [ ]:
# YOUR CODE

Теперь можно заменить значения в колонке `ecology` на заданные нами выше с помощью метода `.replace`:

In [ ]:
data.ecology = data.ecology.replace(to_replace=ecology_dict)

In [ ]:
data.shape

In [ ]:
data

Для добавления новых колонок-признаков воспользуемся методом `.get_dummies`:

In [ ]:
data = pd.get_dummies(data, columns=['sub_area', 'culture_objects_top_25'])

In [ ]:
data.shape

In [ ]:
data.head()

## Обработка мультиколлинеарности
Посмотрим на признаки `children_preschool` и `children_school`. Как правило, подготовка к школе проходит непосредственно в той самой школе, так что количества одних и других заведений обычно оказываются близки.

Посчитайте коэффициенты корреляции между перечисленными колонками:

In [ ]:
# YOUR CODE

Очевидно, что оба столбца нам не нужны: они дают информацию практически об одном и том же. Можно смело удалить один из них (обратите внимание на параметр `inplace` метода `.drop`, позволяющий изменять именно исходную таблицу):

In [ ]:
data.drop('children_preschool', axis=1, inplace=True)

Помимо мультиколлинеарности следует смотреть на корреляцию признаков с целевой переменной. Посмотрите на корреляцию столбца `price_doc` со всеми остальными и оставьте среди них только те, которые оказывают достаточно сильное влияние:

In [ ]:
# YOUR CODE

Оставьте среди первых признаков, соответствующих не категориальным, а исходным числовым значениям, только те, которые не коррелируют между собой:

In [ ]:
# YOUR CODE

Страшный, но, возможно, удобный способ вывести цветную корреляционную таблицу прямо в `pandas` (на семинаре по визуализации расскажем, как сделать проще, не бойтесь).

Содержательная часть только тут `data[data.columns[:10]].corr()`, в остальное можно не вникать, это для красивого отображения.

In [ ]:
data[data.columns[:10]].corr().style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

## Обучение модели
Обучите модель линейной регрессии на полученных данных, посмотрите на полученное качество. Для объективной оценки не забудьте разделить выборку на обучающую и контрольную.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
# YOUR CODE

## Нормализация признаков
Казалось бы, всё уже неплохо. Но давайте посмотрим на разброс признаков: некоторые принимают значения порядка $1000-10000$, а некоторые -- вообще только $0$ или $1$. Это может плохо сказаться на линейной модели, потому что она будет воспринимать более "крупные" признаки как более важные, что не всегда так.

Хорошим тоном является приведение всех признаков к одной шкале. Например, можно привести значения каждого столбца к шкале $[0..1]$ с помощью следующего преобразования:
$$
\frac{x - min(x)}{max(x)}
$$

Сделать это можно как ручками, так и с помощью питона:

In [ ]:
from sklearn.preprocessing import MinMaxScaler

x = data.values  # returns a numpy array
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data = pd.DataFrame(x_scaled, columns=data.columns)

data.head()

Снова обучите модель, уже на новых данных. Сравните качество с тем, что получалось раньше:

In [ ]:
# YOUR CODE

## Интерпретация коэффициентов
Теперь можно посмотреть на значения коэффициентов модели, обратившись к атрибуту `model.coef_`. Чем больше коэффициент по модулю, тем сильнее влияет признак на предсказанное значение.

_Делать такие выводы можно **только** если признаки нормализованы._

In [ ]:
# YOUR CODE